# N-BEATS-Lightning 

**N-BEATS** represents a cutting-edge model, demonstrating the effectiveness of **pure DL architectures** when it comes to time-series forecasting. It surpasses the performance of traditional statistical methods in the *M3* and *M4* competitions. For a more in-depth understanding of the model, refer to: https://arxiv.org/pdf/1905.10437.pdf.

The **milk production** dataset describes a time-series of milk production (in pounds per cow) over 13 years (1962-1974), and there are 156 observations. We use the first 80% of the observations for training and testing various models while holding back the remaining observations for validating the final model.


### Summary Statistics and Visualizations 

We begin by reading the data and summarizing the variables.

In [1]:
#%% Import necessary libraries
from lightningnbeats import NBeatsNet
from lightningnbeats.loaders import *
import pandas as pd
import lightning.pytorch as pl
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch import loggers as pl_loggers
from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
print("CUDA Available: ",torch.cuda.is_available())
tqdm.pandas()
%matplotlib inline

import tensorboard
import warnings
warnings.filterwarnings('ignore')

# Load the milk.csv dataset
milk = pd.read_csv('../src/lightningnbeats/data/milk.csv', index_col=0)
milkval = milk.values.flatten() # flat numpy array
milk.head()

CUDA Available:  True


,milk_production_pounds
month,
1962-01,589
1962-02,561
1962-03,640
1962-04,656
1962-05,727


The dataset consists of two columns:

- *month:*  The month during which the data was collected, starting from January 1962.
- *milk_production_pounds:*   The amount of milk produced in pounds.


In [ ]:
milk.describe()

In [ ]:
# Convert the 'month' column to datetime format
milk['month'] = pd.to_datetime(milk.index)

# Create Time Series Plot
plt.figure(figsize=(8, 6))
plt.plot(milk['month'], milk['milk_production_pounds'], label='Milk Production')
plt.title('Time Series Plot of Milk Production')
plt.xlabel('Time')
plt.ylabel('Milk Production (pounds)')
plt.legend()
plt.grid(True)
plt.show()


The Time Series Plot above illustrates the trend and seasonality in milk production over time.
It's apparent that there's a recurring pattern every year, which suggests a strong seasonal
component. Additionally, there seems to be a general upward trend in milk production over the years.
Next, let's move on to the Seasonal Decomposition to decompose the time series into its trend,
seasonal, and residual components. This will provide a clearer picture of the underlying patterns
in the data.

In [ ]:
milk.set_index('month', inplace=True)

# Perform seasonal decomposition
decomposition = seasonal_decompose(milk['milk_production_pounds'], model='multiplicative')

# Plot the seasonal decomposition again with the corrected attribute name
plt.figure(figsize=(8, 8))
plt.subplot(411)
plt.plot(decomposition.trend, label='Trend')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(decomposition.seasonal, label='Seasonal')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(decomposition.resid, label='Residual')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(milk['milk_production_pounds'], label='Observed')
plt.legend(loc='best')
plt.tight_layout()
plt.show()


The Seasonal Decomposition plot breaks down the time series into three components:

* Trend: Shows a general upward trend in milk production over the years.
* Seasonal: Illustrates a clear seasonal pattern that repeats every year.
* Residual: Contains the residual values after the trend and seasonal components have
been removed.

### Model Specification

Next we will create a PyTorch Lightning DataModule to load the data, and 3 nbeats_lightning models, 2 generic, and 1 interpretable, with which to train the data and then predict.  We will then compare the predictions of the two models
against the actual data and to each other.

In [ ]:
#%% Define the Generic N-Beats Models
# Generic hyperparameters
forecast_length = 6
backcast_length = 4 * forecast_length
batch_size = 64
n_stacks = 6

g_blocks_per_stack = 1
g_width = 512
active_g = False


# Create a simple pytorch dataloader
dm = TimeSeriesDataModule(
  data=milkval,
  batch_size=batch_size,
  backcast_length=backcast_length,
  forecast_length=forecast_length)


stacks = [ 'Generic']
n_stacks = n_stacks//len(stacks)  
stack_types = stacks * n_stacks
no_val=False

g1 = NBeatsNet (
  backcast_length = backcast_length,
  forecast_length = forecast_length, 
  stack_types = stack_types,
  n_blocks_per_stack = 1,
  share_weights = True, 
  thetas_dim = 5,      
  loss = 'SMAPELoss',
  active_g = False,
  latent_dim = 4,
  no_val=no_val,
) 

model_id="".join(stacks)
g1_name = f"{model_id}[{backcast_length},{forecast_length}]-{n_stacks=}-{active_g=}" 
print("Model Name :", g1_name)

# Same model as above, but with g-activation = True.
# g-activation applies an activation funtion to the linear (gb and gf) layers
# which are in the last layer of each block and have as parametes the 
# theta coefficients found in the preceding layer. Application of this activation
# helps the generic model converge faster.

g2 = NBeatsNet (
  backcast_length = backcast_length,
  forecast_length = forecast_length, 
  stack_types = stack_types,
  n_blocks_per_stack = 1,
  share_weights = True, 
  thetas_dim = 5,      
  loss = 'SMAPELoss',
  active_g = True,
  no_val=no_val,
) 
g2_name = f"{model_id}[{backcast_length},{forecast_length}]-{n_stacks=}-{active_g=}" 
print("Model Name :", g2_name)

You will notice that the model `g2` is identical to `g1` except that it has enabled the `active_g` parameter. This parameter when enabled applies the model's activation funtion to the linear funtions (gb and gf) which are found by the network in the last layer of each block using the theta parameters found in the preceding layer. 

You can enable this feature by setting `active_g` to `True`.  Enabling this activation function seems to help the Generic model converge. The parameter `active_g` is not a feature found in the original N-Beats paper.

In [ ]:
# Define the Interpretable N-Beats Model

stacks = ['Trend', 'Seasonality']
# Interpretable Hyperparameters
n_blocks_per_stack = 3
t_width = 256
s_width = 2048


# An Interpretable N-Beats Model, 
#  - 2 stacks 
#     - 3 trend(256) blocks in first stack (default size)
#     - 3 seasonality(2048) in second stack (default size)
interpretable_milkmodel = NBeatsNet(
  stack_types=stacks,
  backcast_length = backcast_length,
  forecast_length = forecast_length, 
  n_blocks_per_stack = n_blocks_per_stack,
  thetas_dim = 5,    
  t_width=t_width,  
  s_width=s_width,
  share_weights = True
)

i_name = f"Interpretable-[{backcast_length}-{forecast_length}]-s[2-{n_blocks_per_stack}]-[{t_width}-{s_width}]" 
print("Model Name :", i_name)

First we train the base Generic model.

In [ ]:
# Define a model checkpoint callback
g1_chk_callback = ModelCheckpoint(
  save_top_k = 1, # save top model
  monitor = "val_loss", # monitor validation loss as evaluation 
  mode = "min",
  filename = "{name}-{epoch:02d}-{val_loss:.2f}",
)

# Define a tensorboard loger

g1_tb_logger = pl_loggers.TensorBoardLogger(save_dir="lightning_logs/", name=g1_name)


# Train the generic model
generic_trainer =  pl.Trainer(
  accelerator='auto' # use GPU if available
  ,max_epochs=500
  ,callbacks=[g1_chk_callback]  
  ,logger=[g1_tb_logger]
)

generic_trainer.fit(g1, datamodule=dm)
generic_trainer.validate(g1, datamodule=dm)

Next we train the generic model with `active_g=True`.  Generally this results in a comparably accurate model in fewer training cycles, when the Generic model converges.  Generic models without an activation on the g layers might not converge at all in fact.   This parameter fixes this shortcoming of the original model. 

In [ ]:
# Train the generic model with active_g

# Define a model checkpoint callback
gact_chk_callback = ModelCheckpoint(
  save_top_k = 2, # save top 2 models
  monitor = "val_loss", # monitor validation loss as evaluation 
  mode = "min",
  filename = "{name}-{epoch:02d}-{val_loss:.2f}",
)
# Define a tensorboard loger
gact_tb_logger = pl_loggers.TensorBoardLogger(save_dir="lightning_logs/", name=g2_name)

generic_act_trainer =  pl.Trainer(
  accelerator='auto' # use GPU if available
  ,max_epochs=500
  ,callbacks=[gact_chk_callback]  
  ,logger=[gact_tb_logger]
)

generic_act_trainer.fit(g2, datamodule=dm)
generic_act_trainer.validate(g2, datamodule=dm)


Next we train the interpretable model.  This model defaults to using only two stacks, per the original paper.  The first stack is the Trend stack, and the second stack is the Seasonal stack.  The Trend stack removes the trend component of the signal, and the Seasonal stack removes the seasonal component of the signal.  The residual component is then used to make the final prediction mirroring the technique used in the traditional statistical method Holt-Winters.

In [ ]:
#%%
# Define a model checkpoint callback
i_chk_callback = ModelCheckpoint(
  save_top_k = 2, # save top 2 models
  monitor = "val_loss", # monitor validation loss as evaluation 
  mode = "min",
  filename = "{name}-{epoch:02d}-{val_loss:.2f}",
)

# Define a tensorboard logger
i_tb_logger = pl_loggers.TensorBoardLogger(save_dir="lightning_logs/", name=i_name)


interpretable_trainer =  pl.Trainer(
  accelerator='auto' # use GPU if available
  ,max_epochs=500
  ,callbacks=[i_chk_callback]  
  ,logger=[i_tb_logger]
)

interpretable_trainer.fit(interpretable_milkmodel, datamodule=dm)
interpretable_trainer.validate(interpretable_milkmodel, datamodule=dm)

Now we can use the trained models to make predictions. We will use the trained models to predict the next quarters worth of milk production.  However, we reuse the validation data in this test example to see how well the models predict the validation data.  In a real world scenario, we would use the trained models to predict the next quarters worth of milk production.

In [ ]:
# Predict
historical_data = torch.tensor(milkval[-backcast_length - forecast_length:- forecast_length], dtype=torch.float32).view(1, -1)

# Create forecasting dataloader
predict_dataset = ForecastingDataset(historical_data)
predict_dataloader = DataLoader(predict_dataset, batch_size=1)

# The model uses a ptl trainer to predict
g1_predictions = generic_trainer.predict(g1, dataloaders=predict_dataloader)
g2_predictions = generic_trainer.predict(g2, dataloaders=predict_dataloader)
interpretable_predictions = interpretable_trainer.predict(interpretable_milkmodel, dataloaders=predict_dataloader)

# The trainer returns a list of dictionaries, one for each dataloader passed to trainer.predict.  We only want the first and only one.
g1_predictions = g1_predictions[0].squeeze()
g2_predictions = g2_predictions[0].squeeze()
interpretable_predictions = interpretable_predictions[0].squeeze()


Set the date range of the precicitons to be the last quarter of the validation data. Also we will only plot a subset of the the data for ease of viewing.

In [ ]:
#%%
predicted_months = pd.date_range(start="1975-06", periods=forecast_length, freq='M').strftime('%Y-%m')
g1_df = pd.DataFrame(g1_predictions, index=predicted_months)
g2_df = pd.DataFrame(g2_predictions, index=predicted_months)


# Plot a subset of the data
interpretable_df = pd.DataFrame(interpretable_predictions, index=predicted_months)
milk_subset = milk[milk.index > '1971-01']
milk_subset.index=milk_subset.index.strftime('%Y-%m')

Use Matplotlib to plot the predictions of the models against the actual data.

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(milk_subset.index, milk_subset['milk_production_pounds'], marker='o', linestyle='-', color='b')
plt.plot(g1_df.index, g1_df, marker='o', linestyle='-', color='g')
plt.plot(g2_df.index, g2_df, marker='o', linestyle='-', color='lightgreen')
plt.plot(interpretable_df.index, interpretable_df, marker='o', linestyle='-', color='indigo')
plt.title('Milk Production Over Time (1971-1975)', fontsize=14)
plt.xlabel('Time (Year-Month)', fontsize=12)
plt.ylabel('Milk Production (lbs)', fontsize=12)
plt.grid(True)
plt.xticks([])
plt.tight_layout()
plt.show()
